Установка зависимостей

In [ ]:
!pip install findspark pyspark

Конфигурация переменных для запуска

In [1]:
accesskey="YCAJEsF9oa2x1Nc9M-K6LvOQv"
secretkey="YCNfV4qnbiRx7zaZ5uzsHB4OoxwdQZa6fC3I1qGD"

Инициализация сессии спарк

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TransactionValidation") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.access.key", accesskey) \
    .config("spark.hadoop.fs.s3a.secret.key", secretkey) \
    .config("spark.hadoop.fs.s3a.endpoint", "storage.yandexcloud.net") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

Указание пути до данных:

In [4]:
data_path = "s3a://qwertyytr/"

Чтение данных с облака:

In [5]:
df = spark.read.option("comment", "#").option("timestampFormat", "ууyy-MM-dd HH:mm:ss") \
.schema("transaction_id LONG, tx_datetime STRING, customer_id INT, terminal_id INT, tx_amount DOUBLE, tx_time_seconds LONG, tx_time_days LONG, tx_fraud INT, tx_fraud_scenario INT") \
.csv(data_path)

Вывод первых 20 значений датафрейма:

In [6]:
df

transaction_id,tx_datetime,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_time_days,tx_fraud,tx_fraud_scenario
0,2019-08-22 06:51:03,0,711,70.91,24663,0,0,0
1,2019-08-22 05:10:37,0,0,90.55,18637,0,0,0
2,2019-08-22 19:05:33,0,753,35.38,68733,0,0,0
3,2019-08-22 07:21:33,0,0,80.41,26493,0,0,0
4,2019-08-22 09:06:17,1,981,102.83,32777,0,0,0
5,2019-08-22 18:41:25,3,205,34.2,67285,0,0,0
6,2019-08-22 03:12:21,3,0,47.2,11541,0,0,0
7,2019-08-22 22:36:40,6,809,139.39,81400,0,0,0
8,2019-08-22 17:23:29,7,184,87.24,62609,0,0,0
9,2019-08-22 21:09:37,8,931,61.7,76177,0,0,0


Вывод первых 20 найденных дубликатов:

In [7]:
df.exceptAll(df.dropDuplicates(['transaction_id']))

transaction_id,tx_datetime,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_time_days,tx_fraud,tx_fraud_scenario
11100352,2019-08-29 10:35:09,88258,585,60.39,642909,7,0,0
32923652,2019-09-12 11:31:23,21762,259,103.54,1855883,21,0,0
10943542,2019-08-28 11:02:45,987639,757,25.89,558165,6,0,0
17114470,2019-09-01 14:42:02,929096,143,24.75,916922,10,0,0
4211972,2019-08-24 16:09:22,689205,123,11.02,230962,2,0,0
2826594,2019-08-23 16:26:51,804638,0,27.45,145611,1,0,0
9771261,2019-08-28 15:22:01,239654,929,91.17,573721,6,0,0
44000745,2019-09-19 12:28:33,92931,37,19.48,2464113,28,0,0
34302649,2019-09-12 07:45:44,901706,139,34.23,1842344,21,0,0
13685700,2019-08-30 13:10:59,739253,0,17.99,738659,8,0,0


Удаление найденных дубликатов:

In [ ]:
df=df.dropDuplicates(['transaction_id'])

Вывод первых 20 найденных пропусков и отрицательных значений:

In [8]:
from pyspark.sql.functions import col
df.exceptAll(df.filter(
    (col("transaction_id") >= 0) &
    (col("customer_id") >= 0) &
    (col("terminal_id") >= 0) &
    (col("tx_amount") >= 0) &
    (col("tx_time_seconds") >= 0) &
    (col("tx_time_days") >= 0) &
    (col("tx_fraud") >= 0) &
    (col("tx_fraud_scenario") >= 0)
))

transaction_id,tx_datetime,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_time_days,tx_fraud,tx_fraud_scenario
47392780,2019-09-21 07:49:27,-999999,481,74.61,2620167,30,0,0
48484096,2019-09-21 06:34:09,953746,null,48.48,2615649,30,1,2
52992134,2019-09-24 07:35:00,830649,null,22.15,2878500,33,1,2
81500036,2019-10-13 12:11:40,31187,null,34.79,4536700,52,0,0
92885348,2019-10-20 02:30:11,300876,null,22.79,5106611,59,0,0
87456235,2019-10-16 17:21:55,834765,null,79.1,4814515,55,1,2
89657464,2019-10-18 12:02:08,239253,null,31.48,4968128,57,0,0
56534717,2019-09-27 13:07:00,-999999,897,45.6,3157620,36,0,0
78166913,2019-10-10 16:30:33,904236,null,42.6,4293033,49,0,0
81537708,2019-10-13 08:57:41,55265,null,44.16,4525061,52,0,0


Удаление найденных пропусков и отрицательных значений:

In [ ]:
from pyspark.sql.functions import col
df=df.filter(
    (col("transaction_id") >= 0) &
    (col("customer_id") >= 0) &
    (col("terminal_id") >= 0) &
    (col("tx_amount") >= 0) &
    (col("tx_time_seconds") >= 0) &
    (col("tx_time_days") >= 0) &
    (col("tx_fraud") >= 0) &
    (col("tx_fraud_scenario") >= 0)
)

Вывод первых 20 найденных неправильных дат:

In [9]:
from pyspark.sql.functions import unix_timestamp
df.exceptAll(df.withColumn("tx_datetime", unix_timestamp("tx_datetime", "yyyy-MM-dd HH:mm:ss").cast("timestamp")))

transaction_id,tx_datetime,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_time_days,tx_fraud,tx_fraud_scenario
8895993,2019-08-27 24:00:00,680343,312,119.13,518400,5,0,0
32550907,2019-09-11 24:00:00,783981,132,9.71,1814400,20,0,0
13334429,2019-08-30 24:00:00,515724,971,6.82,777600,8,1,2
86040345,2019-10-15 24:00:00,931059,936,4.42,4752000,54,0,0
32949123,2019-09-12 24:00:00,37940,554,55.54,1900800,21,0,0
69858189,2019-10-05 24:00:00,600054,352,9.34,3888000,44,0,0
76299966,2019-10-09 24:00:00,711712,226,11.2,4233600,48,0,0
24144197,2019-09-06 24:00:00,415580,857,85.66,1382400,15,1,2
36545600,2019-09-14 24:00:00,334010,527,44.97,2073600,23,0,0
39761540,2019-09-16 24:00:00,386904,824,34.95,2246400,25,0,0


Удаление найденных неправильных дат:

In [ ]:
from pyspark.sql.functions import unix_timestamp
df=df.withColumn("tx_datetime", unix_timestamp("tx_datetime", "yyyy-MM-dd HH:mm:ss").cast("timestamp"))

Функция для удаления выбросов:

In [12]:
from pyspark.sql.functions import col
def remove_outliers_iqr(df, columns):
    for col_name in columns:
        quantiles = df.approxQuantile(col_name, [0.25, 0.75], 0.05)
        q1 = quantiles[0]
        q3 = quantiles[1]
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        df = df.filter((col(col_name) >= lower_bound) & (col(col_name) <= upper_bound))
    return df

Вывод первых 20 найденных выбросов:

In [13]:
df.exceptAll(remove_outliers_iqr(df,["customer_id", "terminal_id", "tx_time_seconds","transaction_id", "tx_time_days"]))

transaction_id,tx_datetime,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_time_days,tx_fraud,tx_fraud_scenario
5190194,2019-08-25 04:56:34,313745,89518096,102.99,276994,3,0,0
5555898,2019-08-25 17:59:23,546992,89518096,24.7,323963,3,0,0
5720560,2019-08-25 11:15:35,652202,89518096,47.52,299735,3,0,0
5930732,2019-08-25 11:50:28,785793,89518096,29.21,301828,3,0,0
5937076,2019-08-25 09:06:18,789788,89518096,10.78,291978,3,0,0
6033043,2019-08-25 17:38:57,851126,89518096,106.48,322737,3,0,0
6623137,2019-08-26 11:49:11,228250,89518096,26.24,388151,4,0,0
12406682,2019-08-29 17:34:09,922218,89518096,19.19,668049,7,0,0
13346487,2019-08-30 13:55:20,523525,89518096,7.16,741320,8,0,0
13551395,2019-08-30 10:55:15,654007,89518096,11.65,730515,8,0,0


Удаление найденных выбросов:

In [ ]:
df=remove_outliers_iqr(df,["customer_id", "terminal_id", "tx_time_seconds","transaction_id", "tx_time_days"])

Сортировка id транзакций от меньшего к большему:

In [ ]:
df=df.orderBy("transaction_id")

Вывод первых 20 значений датафрейма:

In [ ]:
df

Запись в bucket:

In [ ]:
df = df.repartition(40)
df.write.mode("overwrite").option("header", "true").parquet("s3a://ducket4/filespark") 

Остановка сессии spark:

In [14]:
spark.stop()